In [1]:
import random

random.seed(25) # set a random seed
copy_raw_training_data = raw_training_data.copy()  # copy the raw training data
shape = copy_raw_training_data.shape  # store dimension of raw training data
num_entries = shape[0] * (shape[1] - 1)  # store number of entries for all independent variables
num_null = int(num_entries * 0.01)  # make 1% of the data be null entries

# randomly select entries and replace them with NaN, excluding the response variable
for _ in range(num_null):
    rand_row, rand_col = random.randint(0, shape[0] - 1) , random.randint(0, shape[1] - 2) # select a random row and column
    copy_raw_training_data.iloc[rand_row, rand_col] = np.nan # store entry as nan
    
# create a list to store all of the entries which are null: (row, column)
null_entries = [(row_index, col_index) 
                for row_index, row in enumerate(copy_raw_training_data.values) 
                for col_index, val in enumerate(row) 
                if pd.isnull(val)]

NameError: name 'raw_training_data' is not defined

In [2]:
# identify the numerical and categorical columns
num_cols = copy_raw_training_data.select_dtypes(include='number').columns
categorical_cols = copy_raw_training_data.select_dtypes(include='object').columns

# impute numerical missing values with median
copy_raw_training_data[num_cols] = copy_raw_training_data[numerical_columns].fillna(copy_raw_training_data[num_cols].median())

# impute categorical missing values with mode
copy_raw_training_data[categorical_columns] = copy_raw_training_data[categorical_columns].fillna(copy_raw_training_data[categorical_columns].mode().iloc[0])

NameError: name 'copy_raw_training_data' is not defined

In [ ]:
# store all of the entries for the original data set
actual_entries_store = []
for row, col in null_entries:
    actual_entries_store.append(raw_training_data.iloc[row, col])

# store all of the imputed entries for first imputation method
imputed1_entries_store = []
for row, col in null_entries:
    imputed1_entries_store.append(copy_raw_training_data.iloc[row, col])